The global offshore wind turbine dataset was posted October 4, 2021 and provides "geocoded information on global offshore wind turbines (OWTs) derived from Sentinel-1 synthetic aperture radar (SAR) time-series images from 2015 to 2019. It identified 6,924 wind turbines comprising of more than 10 nations." This notebook extracts the data from the dataset's Shapefile and transforms it to be uploadable to flourish to make a racing bar chart.
Dataset: https://figshare.com/articles/dataset/Global_offshore_wind_farm_dataset/13280252/5
Flourish Graphic: https://public.flourish.studio/visualisation/7385349/

In [1]:
import shapefile
import csv
import pandas as pd
import numpy as np

path_to_shapefile = "C:\\...\\GOWF_V1.3.shp"
path_to_csv_out = "C:\\...\\OffshoreWind.csv"

Download shapefiles from https://figshare.com/articles/dataset/Global_offshore_wind_farm_dataset/13280252/5

In [2]:
sf = shapefile.Reader(path_to_shapefile)

In [3]:
print( sf)

shapefile Reader
    6924 shapes (type 'POINT')
    6924 records (8 fields)


In [4]:
fields = sf.fields
print(fields)

[('DeletionFlag', 'C', 1, 0), ['centr_lat', 'F', 13, 11], ['centr_lon', 'F', 13, 11], ['continent', 'C', 50, 0], ['country', 'C', 50, 0], ['sea_area', 'C', 50, 0], ['occ_year', 'N', 10, 0], ['occ_month', 'N', 10, 0]]


Read the raw records in the shapefile metadata

In [5]:
print(sf.records())

[Record #0: [9.20576, 105.782, 'Asia', 'Vietnam', 'South China Sea', 2016, 3], Record #1: [9.20389, 105.783, 'Asia', 'Vietnam', 'South China Sea', 2016, 3], Record #2: [9.2066, 105.789, 'Asia', 'Vietnam', 'South China Sea', 2016, 2], Record #3: [9.20865, 105.788, 'Asia', 'Vietnam', 'South China Sea', 2016, 2], Record #4: [9.20774, 105.781, 'Asia', 'Vietnam', 'South China Sea', 2016, 1], Record #5: [9.20935, 105.796, 'Asia', 'Vietnam', 'South China Sea', 2016, 1], Record #6: [9.21071, 105.788, 'Asia', 'Vietnam', 'South China Sea', 2016, 4], Record #7: [9.21174, 105.78, 'Asia', 'Vietnam', 'South China Sea', 2016, 2], Record #8: [9.20978, 105.78, 'Asia', 'Vietnam', 'South China Sea', 2016, 2], Record #9: [9.2114, 105.795, 'Asia', 'Vietnam', 'South China Sea', 2015, 9], Record #10: [9.21348, 105.795, 'Asia', 'Vietnam', 'South China Sea', 2015, 9], Record #11: [9.21461, 105.802, 'Asia', 'Vietnam', 'South China Sea', 2015, 8], Record #12: [9.21253, 105.803, 'Asia', 'Vietnam', 'South China Se

Begin transforming the data to match desired format

In [6]:
header = ['latitude', 'longitude', 'continent', 'country','sea_area','year','month']
df = pd.DataFrame(sf.records(),columns = header)
df['month']=df['month'].apply(lambda x: '{0:0>2}'.format(x))
print(df)

      latitude  longitude continent  country         sea_area  year month
0      9.20576   105.7820      Asia  Vietnam  South China Sea  2016    03
1      9.20389   105.7830      Asia  Vietnam  South China Sea  2016    03
2      9.20660   105.7890      Asia  Vietnam  South China Sea  2016    02
3      9.20865   105.7880      Asia  Vietnam  South China Sea  2016    02
4      9.20774   105.7810      Asia  Vietnam  South China Sea  2016    01
...        ...        ...       ...      ...              ...   ...   ...
6919  65.65020    24.5631    Europe  Finland  Gulf of Bothnia  2015    01
6920  65.65390    24.5205    Europe  Finland  Gulf of Bothnia  2015    01
6921  65.65650    24.5044    Europe  Finland  Gulf of Bothnia  2015    01
6922  65.66130    24.5019    Europe  Finland  Gulf of Bothnia  2015    01
6923  65.66680    24.5044    Europe  Finland  Gulf of Bothnia  2015    01

[6924 rows x 7 columns]


In [7]:
df['yearmonth'] = df.year.map(str) + "/" + df.month.map(str)

In [8]:
df.insert(0, 'ID', df.index)

In [9]:
df

,ID,latitude,longitude,continent,country,sea_area,year,month,yearmonth
0,0,9.20576,105.7820,Asia,Vietnam,South China Sea,2016,03,2016/03
1,1,9.20389,105.7830,Asia,Vietnam,South China Sea,2016,03,2016/03
2,2,9.20660,105.7890,Asia,Vietnam,South China Sea,2016,02,2016/02
3,3,9.20865,105.7880,Asia,Vietnam,South China Sea,2016,02,2016/02
4,4,9.20774,105.7810,Asia,Vietnam,South China Sea,2016,01,2016/01
...,...,...,...,...,...,...,...,...,...
6919,6919,65.65020,24.5631,Europe,Finland,Gulf of Bothnia,2015,01,2015/01
6920,6920,65.65390,24.5205,Europe,Finland,Gulf of Bothnia,2015,01,2015/01
6921,6921,65.65650,24.5044,Europe,Finland,Gulf of Bothnia,2015,01,2015/01
6922,6922,65.66130,24.5019,Europe,Finland,Gulf of Bothnia,2015,01,2015/01


In [10]:
df = df.drop(columns = ['latitude','longitude','sea_area','year','month'])

In [11]:
df

,ID,continent,country,yearmonth
0,0,Asia,Vietnam,2016/03
1,1,Asia,Vietnam,2016/03
2,2,Asia,Vietnam,2016/02
3,3,Asia,Vietnam,2016/02
4,4,Asia,Vietnam,2016/01
...,...,...,...,...
6919,6919,Europe,Finland,2015/01
6920,6920,Europe,Finland,2015/01
6921,6921,Europe,Finland,2015/01
6922,6922,Europe,Finland,2015/01


Take the tranformed table and make a pivot table from the records aggregating them to count the total number of observed wind turbines per month.

In [12]:
table = df.pivot_table(values='ID', index=['continent','country'], columns=['yearmonth'], aggfunc='count')

In [13]:
table

yearmonth                     2015/01  2015/02  2015/03  2015/04  2015/05  \
continent     country                                                       
Asia          China             188.0     14.0      2.0      2.0      4.0   
              Japan               2.0      NaN      NaN      NaN      NaN   
              South Korea         NaN      NaN      NaN      NaN      NaN   
              Vietnam             9.0      NaN      2.0      NaN      1.0   
Europe        Belgium           105.0      3.0      2.0      2.0      4.0   
              Denmark           439.0      5.0      1.0      2.0      2.0   
              Finland             9.0      NaN      NaN      NaN      NaN   
              Germany           605.0     11.0     20.0     19.0     24.0   
              Ireland             5.0      NaN      1.0      NaN      NaN   
              Netherlands        31.0     11.0      8.0     10.0      5.0   
              Spain               NaN      NaN      NaN      NaN      NaN   
              Sweden             63.0      NaN      3.0      1.0      1.0   
              United Kingdom   1140.0     32.0     33.0     25.0     13.0   
North America United States       NaN      NaN      NaN      NaN      NaN   

yearmonth                     2015/06  2015/07  2015/08  2015/09  2015/10  \
continent     country                                                       
Asia          China               4.0      8.0      5.0      6.0      9.0   
              Japan               NaN      NaN      NaN      NaN      NaN   
              South Korea         NaN      NaN      NaN      NaN      NaN   
              Vietnam             1.0      4.0      4.0      6.0      2.0   
Europe        Belgium             5.0      9.0      2.0      6.0      3.0   
              Denmark             4.0      2.0      4.0      4.0      7.0   
              Finland             NaN      NaN      NaN      NaN      NaN   
              Germany            19.0     23.0     16.0     21.0     34.0   
              Ireland             NaN      NaN      NaN      NaN      NaN   
              Netherlands         5.0      5.0      7.0     11.0     15.0   
              Spain               NaN      NaN      NaN      NaN      NaN   
              Sweden              NaN      NaN      NaN      NaN      1.0   
              United Kingdom     14.0     23.0     10.0     12.0     14.0   
North America United States       NaN      NaN      NaN      NaN      NaN   

yearmonth                     ...  2019/02  2019/03  2019/04  2019/05  \
continent     country         ...                                       
Asia          China           ...     37.0     45.0     42.0     50.0   
              Japan           ...      NaN      NaN      NaN      NaN   
              South Korea     ...      2.0      NaN      NaN      NaN   
              Vietnam         ...      NaN      NaN      NaN      NaN   
Europe        Belgium         ...      NaN      NaN      2.0      NaN   
              Denmark         ...      NaN      NaN      NaN      NaN   
              Finland         ...      NaN      NaN      NaN      NaN   
              Germany         ...      NaN      NaN      NaN      NaN   
              Ireland         ...      NaN      NaN      NaN      NaN   
              Netherlands     ...      NaN      NaN      NaN      NaN   
              Spain           ...      NaN      NaN      NaN      NaN   
              Sweden          ...      NaN      NaN      NaN      NaN   
              United Kingdom  ...      7.0      2.0      NaN      NaN   
North America United States   ...      NaN      NaN      NaN      NaN   

yearmonth                     2019/06  2019/07  2019/08  2019/09  2019/10  \
continent     country                                                       
Asia          China              44.0     48.0     29.0     21.0     32.0   
              Japan               NaN      NaN      NaN      NaN      NaN   
              South Korea         NaN      NaN      NaN  

The values in the table represent the total number of NEW wind turbines found in the month. I need the running total and use the "cumsum" method to add the totals up left to right, and then fill in the zeroes.

In [14]:
table=table.cumsum(axis=1).ffill(axis=1).fillna(0)

In [15]:
table

yearmonth                     2015/01  2015/02  2015/03  2015/04  2015/05  \
continent     country                                                       
Asia          China             188.0    202.0    204.0    206.0    210.0   
              Japan               2.0      2.0      2.0      2.0      2.0   
              South Korea         0.0      0.0      0.0      0.0      0.0   
              Vietnam             9.0      9.0     11.0     11.0     12.0   
Europe        Belgium           105.0    108.0    110.0    112.0    116.0   
              Denmark           439.0    444.0    445.0    447.0    449.0   
              Finland             9.0      9.0      9.0      9.0      9.0   
              Germany           605.0    616.0    636.0    655.0    679.0   
              Ireland             5.0      5.0      6.0      6.0      6.0   
              Netherlands        31.0     42.0     50.0     60.0     65.0   
              Spain               0.0      0.0      0.0      0.0      0.0   
              Sweden             63.0     63.0     66.0     67.0     68.0   
              United Kingdom   1140.0   1172.0   1205.0   1230.0   1243.0   
North America United States       0.0      0.0      0.0      0.0      0.0   

yearmonth                     2015/06  2015/07  2015/08  2015/09  2015/10  \
continent     country                                                       
Asia          China             214.0    222.0    227.0    233.0    242.0   
              Japan               2.0      2.0      2.0      2.0      2.0   
              South Korea         0.0      0.0      0.0      0.0      0.0   
              Vietnam            13.0     17.0     21.0     27.0     29.0   
Europe        Belgium           121.0    130.0    132.0    138.0    141.0   
              Denmark           453.0    455.0    459.0    463.0    470.0   
              Finland             9.0      9.0      9.0      9.0      9.0   
              Germany           698.0    721.0    737.0    758.0    792.0   
              Ireland             6.0      6.0      6.0      6.0      6.0   
              Netherlands        70.0     75.0     82.0     93.0    108.0   
              Spain               0.0      0.0      0.0      0.0      0.0   
              Sweden             68.0     68.0     68.0     68.0     69.0   
              United Kingdom   1257.0   1280.0   1290.0   1302.0   1316.0   
North America United States       0.0      0.0      0.0      0.0      0.0   

yearmonth                     ...  2019/02  2019/03  2019/04  2019/05  \
continent     country         ...                                       
Asia          China           ...   1532.0   1577.0   1619.0   1669.0   
              Japan           ...      4.0      4.0      4.0      4.0   
              South Korea     ...     21.0     21.0     21.0     21.0   
              Vietnam         ...     62.0     62.0     62.0     62.0   
Europe        Belgium         ...    349.0    349.0    351.0    351.0   
              Denmark         ...    560.0    560.0    560.0    560.0   
              Finland         ...     20.0     20.0     20.0     20.0   
              Germany         ...   1524.0   1524.0   1524.0   1524.0   
              Ireland         ...      7.0      7.0      7.0      7.0   
              Netherlands     ...    293.0    293.0    293.0    293.0   
              Spain           ...      2.0      2.0      2.0      2.0   
              Sweden          ...     75.0     75.0     75.0     75.0   
              United Kingdom  ...   2138.0   2140.0   2140.0   2140.0   
North America United States   ...      5.0      5.0      5.0      5.0   

yearmonth                     2019/06  2019/07  2019/08  2019/09  2019/10  \
continent     country                                                       
Asia          China            1713.0   1761.0   1790.0   1811.0   1843.0   
              Japan               4.0      4.0      4.0      4.0      4.0   
              South Korea        21.0     21.0     21.0  

In [16]:
table.to_csv(path_to_csv_out)